<a href="https://colab.research.google.com/github/cbonnin88/RailFlow/blob/main/A_B_Testing_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest, proportion_effectsize
import statsmodels.stats.api as sms

# **Part 1: Power Analysis**
**Question:** How many users do we need to trust the results ?

In [29]:
# 1. Defining the Scenario
baseline_conversion = 0.12 # 12% (Current Control Group)
target_conversion = 0.125 # 12.5% (Expected Variant Group)

In [32]:
# 2. Calculate Effect Size (Standardized difference between two proportions)
effect_size = proportion_effectsize(baseline_conversion, target_conversion)

In [33]:
# 3. Calculate Required Sample Size
# Power = 0.8 (80% chance to detect the lift if its real)
# Alpha = 0.05 (95% confidence level)
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
)

required_n = int(np.ceil(required_n)) # Round up to the nearest whole number

In [34]:
print(f"📊 PART 1: POWER ANALYSIS")
print(f"To detect a lift from {baseline_conversion:.1%} to {target_conversion:.1%}, we need:")
print(f"👉 {required_n:,} users per group.")
print("-" * 50)

📊 PART 1: POWER ANALYSIS
To detect a lift from 12.0% to 12.5%, we need:
👉 67,490 users per group.
--------------------------------------------------


# **Part 2: Hypothesis Testing**

**Question:** We ran the test with N users. Is the result significant?

In [35]:
# 1. Simulate The Experiment Data
# Using the required_n that was just calculated above
n_A = required_n
n_B = required_n

In [37]:
# Simulating the actual number of bookings (Successes)
# In real life, these numbers would come from your database (in my case BigQuery)
successes_A = int(n_A * baseline_conversion)
successes_B = int(n_B * target_conversion)

In [38]:
print(f"Group A (Control): {successes_A:,} bookings / {n_A:,} users")
print(f"Group B (Variant): {successes_B:,} bookings / {n_B:,} users")

Group A (Control): 8,098 bookings / 67,490 users
Group B (Variant): 8,436 bookings / 67,490 users


In [39]:
# 2. Run the Z-Test (Two- Proportion)
count = np.array([successes_A,successes_B])
nobs = np.array([n_A,n_B])

In [40]:
stat, p_value = proportions_ztest(count,nobs, alternative='two-sided')

In [41]:
print(f"\n📉 STATISTICAL RESULTS")
print(f"Z-Score: {stat:.4f}")
print(f"P-Value: {p_value:.4f}")


📉 STATISTICAL RESULTS
Z-Score: -2.8061
P-Value: 0.0050


In [42]:
# 3. Final Decision
alpha = 0.05 # Standard threshold

In [43]:
if p_value < alpha:
  print('\n✅ RESULT: STATISTICALLY SIGNIFICANT!')
  print('Conclusion: The 0.5% lift is real. We should launch the feature.')
else:
  print("\n❌ RESULT: NOT SIGNIFICANT.")
  print("Conclusion: We cannot distinguish this result from random luck.")


✅ RESULT: STATISTICALLY SIGNIFICANT!
Conclusion: The 0.5% lift is real. We should launch the feature.
